In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
collegeboard = pd.DataFrame()
for year in range(18, 23):  # range is exclusive at the end
    filename = f"/Users/adamcartwright/ncerdc/Student Data/Testing Data/collegeboard{year}pub.sas7bdat"
    try:
        # Read SAS file
        df = pd.read_sas(filename)

        # Keep only relevant columns
        df.columns = df.columns.str.lower()

        # Drop missing values

        
        # Append to the main DataFrame
        collegeboard = pd.concat([collegeboard, df], ignore_index=True)
        print("{year} done")

    except FileNotFoundError:
        print(f"File {filename} not found. Skipping.")
    except KeyError:
        print(f"Required columns not found in {filename}. Skipping.")

{year} done
{year} done
{year} done
{year} done
{year} done
